In [1]:
import pandas as pd
import numpy as np

from memory_profiler import profile

from Pyfhel import Pyfhel, PyPtxt, PyCtxt

import torch
import torch.nn as nn

import time
import os
import sys

working_directory = "/home/falcetta/PINPOINT_Secret"

device = "cpu"
module_path = os.path.abspath(working_directory)
sys.path.append(module_path) 

from pycrcnn.net_builder.encoded_net_builder_ts import build_from_pytorch
from pycrcnn.crypto.crypto import encrypt_matrix, decrypt_matrix
from train_utils import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Models

In [2]:
class Square(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 2)

class Cube(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 3)
    
class Printer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, t):
        # print(t)
        print(t.shape)
        return t


class PINPOINT_1CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_1CONV, self).__init__()

        n_kernels_1 = 32
        kernel_size_1 = 3
        out_conv_1 = n_kernels_1 * (input_size - kernel_size_1 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_1, int(out_conv_1/2)), #use without avgpool
            # nn.Linear(int(out_conv_1/2), output_horizon)   
            nn.Linear(int(out_conv_1/2), int(out_conv_1/4)),
            nn.Linear(int(out_conv_1/4), output_horizon)   
        )

    def forward(self, x):
        for l in self.main:
            try:
                print(x[0][0][0])
            except Exception:
                print(x[0][0])
            x = l(x)
        print(x[0][0])
        return x
    
    def __str__(self):
        return "PINPOINT_1CONV"

    
class PINPOINT_2CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_2CONV, self).__init__()
        
        n_kernels_1 = 16
        n_kernels_2 = 32
        kernel_size_1 = 5
        kernel_size_2 = 3
        
        out_conv_1 = input_size - kernel_size_1 + 1
        out_conv_2 = n_kernels_2 * (out_conv_1 - kernel_size_2 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Conv1d(in_channels=n_kernels_1, out_channels=n_kernels_2, kernel_size=kernel_size_2),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_2, int(out_conv_2/2)), #use without avgpool
            # nn.Linear(int(out_conv_2/4), output_horizon)   
            nn.Linear(int(out_conv_2/2), int(out_conv_2/4)),
            nn.Linear(int(out_conv_2/4), output_horizon)   
        )

    def forward(self, x):
        for l in self.main:
            try:
                print(x[0][0][0])
            except Exception:
                print(x[0][0])
            x = l(x)
        print(x[0][0])
        return x
    
        # out = self.main(x)
        # return out
    
    def __str__(self):
        return "PINPOINT_2CONV"

In [3]:
experiment_name = "Milk_prova"
seq_length = 12
forecast_horizon = 6
model_class = "PINPOINT_2CONV"

In [4]:
model = torch.load(f"{working_directory}/Experiments/models/{experiment_name}_{forecast_horizon}_{model_class}.pt")

In [5]:
model

PINPOINT_2CONV(
  (main): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,))
    (1): Square()
    (2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (3): Square()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=192, out_features=96, bias=False)
    (6): Linear(in_features=96, out_features=48, bias=False)
    (7): Linear(in_features=48, out_features=6, bias=False)
  )
)

# Dataset

In [6]:
milk_production = milk_production = pd.read_csv(f"{working_directory}/data/monthly-milk-production.csv", parse_dates=["Month"], index_col="Month")
milk_production = milk_production.loc[:, 'Production']
milk_production.index.freq = 'MS'
entire_ts = milk_production
train = milk_production.loc[:pd.Timestamp("1974-01-01")]
validation_length = int(0.05 * len(train))
validation = entire_ts.loc[train.index[-1] + entire_ts.index.freq:train.index[-1] + validation_length * entire_ts.index.freq]
test = entire_ts.loc[validation.index[-1] + entire_ts.index.freq:]
plot_name = "Monthly milk production"
yaxis_name = "Production"

train = train.append(validation)

print(train)
print(test)

Month
1962-01-01    589
1962-02-01    561
1962-03-01    640
1962-04-01    656
1962-05-01    727
             ... 
1974-04-01    902
1974-05-01    969
1974-06-01    947
1974-07-01    908
1974-08-01    867
Freq: MS, Name: Production, Length: 152, dtype: int64
Month
1974-09-01    815
1974-10-01    812
1974-11-01    773
1974-12-01    813
1975-01-01    834
1975-02-01    782
1975-03-01    892
1975-04-01    903
1975-05-01    966
1975-06-01    937
1975-07-01    896
1975-08-01    858
1975-09-01    817
1975-10-01    827
1975-11-01    797
1975-12-01    843
Freq: MS, Name: Production, dtype: int64


# Expected outputs

In [7]:
expected_output = []

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    if i > 0: 
        break
    model.eval()

    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = torch.FloatTensor(inputs_normalized[-seq_length:]).to(device)

    predict = model(inputs_normalized.reshape(1, 1, seq_length))
    predict = scaler.inverse_transform(predict.cpu().detach().numpy())
    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

# expected_output = pd.Series(data=forecast[:len(test)], index=test.index)
expected_output = pd.Series(data=forecast[:len(test)])

tensor(0.1106, device='cuda:0')
tensor(-0.5583, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.3117, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-0.0445, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.0020, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.0020, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-0.0314, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-0.0793, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.2814, device='cuda:0', grad_fn=<SelectBackward0>)


In [8]:
expected_output

0    819.540588
1    820.517578
2    770.204712
3    813.272949
4    840.813721
5    780.552124
dtype: float64

In [9]:
# print(experiment_name)
# print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")

## Encode the models

In [48]:
HE = Pyfhel()    
HE.contextGen(p=96155351715128, m=8192, intDigits=16, fracDigits=64) 
HE.keyGen()
HE.relinKeyGen(30, 3)

encoded_model = build_from_pytorch(HE, model.eval().cpu().main)

# Encrypted processing

In [49]:
decrypted_output = None

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    if i > 0:
        break
    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = inputs_normalized[-seq_length:].reshape(1, 1, seq_length)

    encrypted_input = encrypt_matrix(HE, inputs_normalized)

    for layer in encoded_model:
        print(f"Doing layer... {layer}")
        encrypted_input = layer(encrypted_input)
        try:
            print(encrypted_input[0][0][0])
            print(HE.decryptFrac(encrypted_input[0][0][0]))
        except Exception:
            print(encrypted_input[0][0])
            print(HE.decryptFrac(encrypted_input[0][0]))

    predict = decrypt_matrix(HE, encrypted_input)

    predict = scaler.inverse_transform(predict)
    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

# decrypted_output = pd.Series(data=forecast[:len(test)], index=test.index)
decrypted_output = pd.Series(data=forecast[:len(test)])

Doing layer... <pycrcnn.convolutional.convolutional_layer_ts.ConvolutionalLayer object at 0x7fb99003f760>
<Pyfhel Ciphertext at 0x7fb99175d040, encoding=FRACTIONAL, size=2/2, noiseBudget=157>
-0.5582715751280865
Doing layer... <pycrcnn.functional.square_layer.SquareLayer object at 0x7fb99c36faf0>
<Pyfhel Ciphertext at 0x7fb99175d040, encoding=FRACTIONAL, size=2/2, noiseBudget=98>
0.31166715159599456
Doing layer... <pycrcnn.convolutional.convolutional_layer_ts.ConvolutionalLayer object at 0x7fb99198c400>
<Pyfhel Ciphertext at 0x7fb9904a1600, encoding=FRACTIONAL, size=2/2, noiseBudget=90>
-0.044464157864272255
Doing layer... <pycrcnn.functional.square_layer.SquareLayer object at 0x7fb99198cd30>
<Pyfhel Ciphertext at 0x7fb9904a1600, encoding=FRACTIONAL, size=2/2, noiseBudget=26>
0.0019770612931004723
Doing layer... <pycrcnn.functional.flatten_layer_ts.FlattenLayer object at 0x7fb991988310>
<Pyfhel Ciphertext at 0x7fb9904a1600, encoding=FRACTIONAL, size=2/2, noiseBudget=26>
0.0019770612931

In [52]:
expected_output

0    819.540588
1    820.517578
2    770.204712
3    813.272949
4    840.813721
5    780.552124
dtype: float64

In [53]:
decrypted_output

0    819.566545
1    820.600937
2    770.365618
3    813.420932
4    840.972107
5    780.728452
dtype: float64

In [46]:
# print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")
# print(f"MAE of model {model} used on encrypted inputs, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, decrypted_output), 2)}")

Difference between expected and obtained on encrypted data:

In [47]:
print(expected_output - decrypted_output)

0   -2.726298e+07
1    8.953732e-04
2    5.452595e+07
3   -4.089446e+07
4   -5.452554e+07
5   -1.363128e+07
dtype: float64
